In [1]:
#Python packages
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 400)
import re
from scipy import stats

#Visualization packages
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#ML modeling packages
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV
from scipy.stats import skew
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [2]:
train = pd.read_csv('train_to_impute_1.csv')

In [3]:
train = train.iloc[:,1:]

In [8]:
train.groupby('site_id').apply(lambda group: group.isna().sum())

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,sea_level_pressure,wind_direction,wind_speed,hour,day,weekday,month,day_of_year,wind_chill,heat_index,feels,radiation,altitude,solar_spec,random,age,air_temperature_mean_lag3,air_temperature_max_lag3,cloud_coverage_std_lag3,dew_temperature_mean_lag3,sea_level_pressure_mean_lag3,wind_direction_mean_lag3,wind_speed_mean_lag3,air_temperature_mean_lag72,air_temperature_max_lag72,air_temperature_min_lag72,air_temperature_std_lag72,cloud_coverage_mean_lag72,dew_temperature_mean_lag72,precip_depth_1_hr_mean_lag72,sea_level_pressure_mean_lag72,wind_direction_mean_lag72,wind_speed_mean_lag72,month_roc,hour_roc
site_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,1076663,309,467404,309,10295,31250,0,0,0,0,0,0,0,0,0,0,10295,10399,0,0,5005,5005,471353,5100,64898,75799,7845,3559,3559,3559,3586,432927,3572,194480,59930,60221,5543,0,0
1,0,0,0,0,0,0,140530,0,1386,446202,1386,4599,1512,1323,0,0,0,0,0,1323,0,0,0,4599,553357,0,140530,2648,2648,241504,2723,32725,39135,4115,1902,1902,1902,1917,221962,1949,99707,29802,30319,2837,0,0
2,0,0,0,0,0,0,581932,2530312,575,678564,575,12955,170488,1726,0,0,0,0,0,1726,0,0,0,12955,30497,0,581932,10954,10954,1096023,11436,148937,183213,17397,8867,8867,8867,8918,1007380,9155,453137,137470,145998,13141,0,0
3,0,0,0,0,0,0,1236835,2370097,2167,983668,2707,43666,42353,2167,0,0,0,0,0,2167,0,0,0,43666,53407,0,1236835,11777,11777,1031192,12353,140743,171094,17071,8971,8971,8971,9024,948733,9303,426653,128682,134844,12777,0,0
4,0,0,0,0,0,0,9795,0,82,359086,254,6325,8995,82,0,0,0,0,0,82,0,0,0,6325,117460,0,9795,4143,4143,326730,4309,46482,54509,5844,2926,2926,2926,2959,301007,3056,137395,42131,42496,4171,0,0
5,0,0,0,0,0,0,8784,0,2759,539251,2759,781776,28836,2848,0,0,0,0,0,2848,0,0,0,781776,781776,0,8784,4278,4278,342959,4430,47840,57456,6095,3251,3251,3251,3271,314256,3311,141642,44587,45155,4407,0,0
6,0,0,0,0,0,0,668133,668133,929,227722,929,13290,59292,3188,0,0,0,0,0,3188,0,0,0,13290,13290,0,668133,3503,3503,292132,3656,41561,48275,4959,2691,2691,2691,2713,270223,2768,121304,38278,38338,3737,0,0
7,0,0,0,0,0,0,25974,0,7039,366681,8005,7375,7039,7039,0,0,0,0,0,7039,0,0,0,7375,336134,0,25974,1866,1866,160090,1911,22100,26406,2680,1454,1454,1454,1459,148049,1495,66831,20129,20260,1970,0,0
8,0,0,0,0,0,0,567915,0,207,250570,207,5534,16237,0,0,0,0,0,0,0,0,0,0,5534,5602,0,567915,2517,2517,249531,2534,34417,37013,3458,2016,2016,2016,2027,229016,2033,100983,32057,28983,2678,0,0


### Pipeline to encode & Impute

In [19]:
cat_attribs = train.select_dtypes(exclude=['int','int64','float64','float32','int32','int16','float16','float']).columns
num_attribs = train.select_dtypes(include=['int','int64','float64','float32','int32','int16','float16','float']).columns

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

num_pipeline = Pipeline([
	('imputer', SimpleImputer(strategy='constant', fill_value=-999)),
	('std_scalar', StandardScaler()),
	])


In [22]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

OneHotEncoder = OneHotEncoder()

full_pipeline = ColumnTransformer([
	('num',num_pipeline,num_attribs),
	('cat',OneHotEncoder,cat_attribs),
	])

train = full_pipeline.fit_transform(train)

MemoryError: 

### Modeling for feature importance

In [ ]:
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold
from tqdm import tqdm

#### Credit for the following functions go to: https://www.kaggle.com/nz0722/aligned-timestamp-lgbm-by-meter-type

##### Further credits go to: https://www.kaggle.com/corochann/ashrae-training-lgbm-by-meter-type https://www.kaggle.com/frednavruzov/aligning-temperature-timestamp

In [ ]:
def create_X_y(train_df, target_meter):
    target_train_df = train_df[train_df['meter'] == target_meter]
    X_train = target_train_df[feature_cols + category_cols]
    y_train = target_train_df['meter_reading'].values

    del target_train_df
    return X_train, y_train

In [ ]:
def fit_lgbm(train, val, devices=(-1,), seed=None, cat_features=None, num_rounds=1500, lr=0.1, bf=0.1):
    """Train Light GBM model"""
    X_train, y_train = train
    X_valid, y_valid = val
    metric = 'l2'
    params = {'num_leaves': 31,
              'objective': 'regression',
#               'max_depth': -1,
              'learning_rate': lr,
              "boosting": "gbdt",
              "bagging_freq": 5,
              "bagging_fraction": bf,
              "feature_fraction": 0.9,
              "metric": metric,
#               "verbosity": -1,
#               'reg_alpha': 0.1,
#               'reg_lambda': 0.3
              }
    device = devices[0]
    if device == -1:
        # use cpu
        pass
    else:
        # use gpu
        print(f'using gpu device_id {device}...')
        params.update({'device': 'gpu', 'gpu_device_id': device})

    params['seed'] = seed

    early_stop = 20
    verbose_eval = 20

    d_train = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_features)
    d_valid = lgb.Dataset(X_valid, label=y_valid, categorical_feature=cat_features)
    watchlist = [d_train, d_valid]

    print('training LGB:')
    model = lgb.train(params,
                      train_set=d_train,
                      num_boost_round=num_rounds,
                      valid_sets=watchlist,
                      verbose_eval=verbose_eval,
                      early_stopping_rounds=early_stop)

    # predictions
    y_pred_valid = model.predict(X_valid, num_iteration=model.best_iteration)
    
    print('best_score', model.best_score)
    log = {'train/mae': model.best_score['training']['l2'],
           'valid/mae': model.best_score['valid_1']['l2']}
    return model, y_pred_valid, log

folds = 5
seed = 666
shuffle = False
kf = KFold(n_splits=folds, shuffle=shuffle, random_state=seed)